In [6]:
import PyPDF2

def delete_pdf_page(input_path, output_path, page_number):
    with open(input_path, 'rb') as input_file:
        reader = PyPDF2.PdfFileReader(input_file)
        writer = PyPDF2.PdfFileWriter()

        num_pages = reader.numPages

        if page_number < 0 or page_number >= num_pages:
            print(f"Invalid page number. The PDF file has {num_pages} pages.")
            return

        for current_page in range(num_pages):
            if current_page != page_number:
                page = reader.getPage(current_page)
                writer.addPage(page)

        with open(output_path, 'wb') as output_file:
            writer.write(output_file)

        print(f"Page {page_number + 1} deleted successfully.")

# 示例用法
input_path = 'input.pdf'  # 输入PDF文件路径
output_path = 'output.pdf'  # 输出PDF文件路径
page_number_to_delete = 2  # 要删除的页码（从0开始）

delete_pdf_page(input_path, output_path, page_number_to_delete)



ModuleNotFoundError: No module named 'PyPDF2'

In [1]:
from langchain_community.chat_models import ChatZhipuAI
import os
os.environ["ZHIPUAI_API_KEY"] = ""
model = ChatZhipuAI(model="glm-4",streamer = True)

In [2]:
# 以下的格式就可以让模板有大括号
QA_PAIRS_SYSTEM_PROMPT = """  
<Context></Context> 标记中是一段文本，学习和分析它，并整理学习成果：  
- 根据不少于200字的文本提出问题并给出每个问题的答案。  
- 答案需详细完整，尽可能保留原文描述，不少于150字。  
- 答案可以包含普通文字、链接、代码、表格、公示、媒体链接等 Markdown 元素。   
- 中文问题与答案
"""

QA_PAIRS_HUMAN_PROMPT = """  
请按以下json格式整理学习成果返回给我，不需要输出多余的话，符合json解析格式。:  
<Context>  
文本  
</Context>  
[  
{{"instruction": "问题1","output":"答案1"}},  
{{"instruction": "问题2","output":"答案2"}},  
]  
------  
  
我们开始吧!  
  
<Context>  
{text}  
<Context/>  
"""


In [3]:
import re
def split_document(filepath):  
	loader = UnstructuredFileLoader(filepath)  
	text_spliter = RecursiveCharacterTextSplitter(  
		chunk_size=2000,  
		chunk_overlap=64  
	)  
	documents = loader.load_and_split(text_spliter)  
	return documents

def find_list(text):

    # print(text.content)
    # 正则表达式模式
    pattern = r"\[.*?\]"
    
    # 使用 re.search 提取第一个匹配
    match = re.search(pattern, text.content, re.DOTALL)
    
    if match:
        # 提取到的列表字符串
        list_str = match.group(0)
        return list_str  
    else:
        return []


In [4]:
import json  
from typing import List  
  
from tqdm.auto import tqdm  
from langchain_core.prompts import ChatPromptTemplate  
from langchain_core.pydantic_v1 import BaseModel, Field  
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import UnstructuredFileLoader  
from langchain_text_splitters import RecursiveCharacterTextSplitter

prompt = ChatPromptTemplate.from_messages([  
	("system", QA_PAIRS_SYSTEM_PROMPT),  
	("human", QA_PAIRS_HUMAN_PROMPT)  
])
class QaPair(BaseModel):  
	instruction: str = Field(description='问题内容')  
	output: str = Field(description='问题的回答')  

class QaPairs(BaseModel):  
	qas: List[QaPair] = Field(description='问答对列表')

parser = JsonOutputParser(pydantic_object=QaPairs)

chain = prompt | model | RunnableLambda(find_list) | parser

documents = split_document('poem/4.pdf')  

with open(f'dataset.json', 'w', encoding='utf-8') as f:  
    datas = []  
    
    
    for doc in tqdm(documents):  
        
        if doc.page_content:
            # print(doc.page_content)
            out = chain.invoke({'text': str(doc.page_content)})
            out = [input_dict for input_dict in out if isinstance(input_dict, dict)] 
            # print(True)
            
            datas += out
    fulldatas = []
    for data in datas:
        data['input'] = ''
        data['history'] = []
        fulldatas.append(data)
    f.write(json.dumps(fulldatas, ensure_ascii=False))  
  



  0%|          | 0/5 [00:00<?, ?it/s]